In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
root = '/Volumes/Data2/RST/notebook/'

In [3]:
period = 'MD'
trial = 0

In [4]:
path_cost = pd.read_csv(period + '_segment-cost.csv')

In [5]:
walk_links = pd.read_csv(root + 'GIS/int_tts_walk_time.csv')
walk_links['duration'] = round(walk_links['duration']/60).astype(int)
walk_links['gta06'] = walk_links['gta06'] + 1000

In [6]:
walk_o = walk_links.rename(columns = {'gta06': 'INT_ID_o', 'INT_ID':'INT_ID_d', 'duration':'cost'})
walk_d = walk_links.rename(columns = {'gta06': 'INT_ID_d', 'INT_ID':'INT_ID_o', 'duration':'cost'})

walk = walk_o.append(walk_d).copy()
walk['walk'] = True

In [7]:
cost = path_cost.drop(columns = ['std'])
cost['walk'] = False
cost = cost.append(walk)
cost

,INT_ID_o,INT_ID_d,cost,walk
0,100.0,101.0,1.321429,False
1,100.0,151.0,1.819672,False
2,100.0,13467163.0,1.000000,False
3,100.0,13467239.0,2.000000,False
4,100.0,13467593.0,3.000000,False
...,...,...,...,...
37157,13467412.0,1081.0,24.000000,True
37158,13467854.0,1081.0,6.000000,True
37159,13466312.0,1081.0,22.000000,True
37160,13467197.0,1081.0,29.000000,True


In [8]:
list_dir = os.listdir(period + '/' + str(trial))

In [9]:
file_list = []
for file in list_dir:
    if file.split('_')[0] != 'int-path':
        continue
    file_list.append(file)

In [20]:
table = []
i = 0

for file in file_list:
    origin = file.split('_')[1]
    dest = file.split('_')[2].split('.')[0]
    
    try:
        df_o = pd.read_csv(period + '/' + str(trial) + '/' + file)
        
    except:
        print(trial, origin, dest)
        continue
    
    if len(df_o) == 0:
        continue
    
    df_o['sequence'] = df_o.groupby('path').cumcount()
    # creating links from int list
    df_d = df_o.copy()
    df_d['sequence'] = df_d['sequence'] - 1
    df_d = df_d[df_d['sequence'] >= 0]
    df_o = df_o.rename(columns = {'INT_ID':'INT_ID_o'})
    df_d = df_d.rename(columns = {'INT_ID':'INT_ID_d'})
    path_edges = df_o.merge(df_d).sort_values(by = ['path', 'sequence'])

    path_edges = path_edges.merge(cost, how = 'left')

    # all walking
    walk_path = path_edges.groupby('path').count()[['sequence']].copy()
    walk_path['walk_links'] = path_edges.groupby('path')['walk'].sum()

    walk_path_num = len(walk_path[walk_path['sequence'] == walk_path['walk_links']])

    # unique edges
    edges = path_edges[['INT_ID_o', 'INT_ID_d', 'cost', 'walk']].drop_duplicates().copy()


    edges = edges[edges['walk'] == False]
    edges['cost'] = edges['cost'].fillna(0)
    
    if len(edges) == 0:
        continue

    num = 0 
    denom = 0
    gamma = 0

    if walk_path_num > 0:
        n_routes = (path_edges['path'].max() + 1) - walk_path_num + 1
    else:
        n_routes = path_edges['path'].max() + 1
        
    
    if n_routes == 1:
        gamma = 1
    else:
        for index, row in edges.iterrows():

            int_o = row['INT_ID_o']
            int_d = row['INT_ID_d']

            link_cost = row['cost']
            n_a = len(path_edges[(path_edges['INT_ID_o'] == int_o) & (path_edges['INT_ID_d'] == int_d)])

            iteration = (np.log(n_a) * link_cost) 
            denom = denom + np.log(n_routes) * link_cost
            num = num + iteration
        try:
            gamma = 1 - num/denom
        except:
            print(origin, dest)
            break
    if walk_path_num > 0:

        eff_paths = 1 + 1 + gamma * (n_routes - 1 - 1)

    else:

        eff_paths = 1 + gamma * (n_routes - 1)  
        
    if i%500 == 0:
        print(i)
    
    i = i + 1
    
    if n_routes == 0:
        table.append([period, trial, origin, dest, 1, n_routes, 0, 0])
    else:
        table.append([period, trial, origin, dest, gamma, n_routes, walk_path_num, eff_paths])
    

0


/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_1476/2335110093.py:62: RuntimeWarning: divide by zero encountered in log
  denom = denom + np.log(n_routes) * link_cost
/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_1476/2335110093.py:62: RuntimeWarning: invalid value encountered in double_scalars
  denom = denom + np.log(n_routes) * link_cost


500
1000
1500
2000


In [21]:
df = pd.DataFrame.from_records(table, columns = ['period', 'trial', 'origin', 'destination', 'gamma', 'paths', 'walk_paths', 'eff_paths'])

In [22]:
df

,period,trial,origin,destination,gamma,paths,walk_paths,eff_paths
0,MD,0,1001,1253,1.000000,1,2,1.000000
1,MD,0,1002,1072,0.461864,55,0,25.940675
2,MD,0,1014,1025,0.700307,8,6,5.902152
3,MD,0,1014,1038,0.754940,19,7,14.588924
4,MD,0,1015,1049,1.000000,1,0,1.000000
...,...,...,...,...,...,...,...,...
2032,MD,0,1620,1559,1.000000,0,0,0.000000
2033,MD,0,1620,1612,NaN,0,0,NaN
2034,MD,0,1624,1046,0.347193,81,0,28.775410
2035,MD,0,1624,1542,0.400433,54,0,22.222968


In [23]:
df[df['gamma'].isna()]

,period,trial,origin,destination,gamma,paths,walk_paths,eff_paths
93,MD,0,1026,1046,NaN,0,0,NaN
280,MD,0,1051,1110,NaN,0,0,NaN
286,MD,0,1051,1269,NaN,0,0,NaN
428,MD,0,1069,1150,NaN,0,0,NaN
466,MD,0,1074,1145,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...
1933,MD,0,1575,1561,NaN,0,0,NaN
1934,MD,0,1577,1490,NaN,0,0,NaN
1955,MD,0,1585,1494,NaN,0,0,NaN
1991,MD,0,1596,1595,NaN,0,0,NaN


In [27]:
df[df['gamma'] <0]

,period,trial,origin,destination,gamma,paths,walk_paths,eff_paths
24,MD,0,1019,1090,-0.002389,4,0,0.992833
35,MD,0,1021,1090,-0.110892,3,0,0.778217
40,MD,0,1021,1161,-0.010521,5,0,0.957916
42,MD,0,1021,1211,-0.447039,2,0,0.552961
114,MD,0,1036,1043,-0.056949,3,0,0.886103
...,...,...,...,...,...,...,...,...
1798,MD,0,1526,1216,-0.099384,3,0,0.801232
1799,MD,0,1526,1257,-0.383529,2,0,0.616471
1811,MD,0,1531,1279,-0.123864,2,0,0.876136
1913,MD,0,1564,1233,-0.070595,4,0,0.788215


In [26]:
for i in range(0,6,1):
    print(i)

0
1
2
3
4
5
